In [1]:
import numpy as np
import pandas as pd
import cv2
from skimage import io

In [2]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub


In [3]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub

In [4]:
m = hub.KerasLayer('https://tfhub.dev/google/aiy/vision/classifier/food_V1/1')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-04-05 03:03:34.675322: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [11]:
inputFood_url = "foodimage2.jpeg"
labelmap_url = "aiy_food_V1_labelmap.csv"
input_shape = (224, 224)


In [12]:
image = np.asarray(io.imread(inputFood_url), dtype="float")
image = cv2.resize(image, dsize=input_shape, interpolation=cv2.INTER_CUBIC)
# Scale values to [0, 1].
image = image / image.max()
# The model expects an input of (?, 224, 224, 3).
images = np.expand_dims(image, 0)

In [13]:
output = m(images)
predicted_index = output.numpy().argmax()
classes = list(pd.read_csv(labelmap_url)["name"])
print("Prediction: ", classes[predicted_index])

Prediction:  Waffle
